In [ ]:

import osmnx as ox
import folium
import pandas as pd
import numpy as np
import utils
import folium_utils

# from IPython.display import display
# pd.set_option('display.max_rows', None)

# # 峠道
# graph = ox.graph_from_point(center_point=(35.434319, 136.954315)
#                                 , network_type='drive'
#                                 , dist=3000
#                                 , simplify=True
#                                 , retain_all=True
#                                 , custom_filter='["highway"~"tertiary"]')

latitude_start = 35.3527512
longitude_start = 136.9226986
latitude_end = 34.9261989
longitude_end = 137.3264474
graph = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
                           south=min(latitude_start, latitude_end),
                           east=max(longitude_start, longitude_end),
                           west=min(longitude_start, longitude_end),
                           network_type='drive',
                           simplify=True,
                           retain_all=True,
                           custom_filter='["highway"~"tertiary|secondary|primary|trunk"]["lanes"=2]')

# グラフデータをGeoDataFrameに変換  
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

# エッジの長さから評価する。
lower_bound_meter = 200
upper_bound_meter = 2000
gdf_edges['length_evaluation'] = np.where(gdf_edges['length'] <= lower_bound_meter, 0,
                                   np.where(gdf_edges['length'] >= upper_bound_meter, 1,
                                            (gdf_edges['length'] - lower_bound_meter) / (upper_bound_meter - lower_bound_meter)))

# エッジからジオメトリーのポイント数を取得する(ポイントが多い = Rがついている道?, エッジが重なっている所もポイント数が多くなるので注意(後々で別の評価関数を実装する予定))
gdf_edges['point_cnt'] = gdf_edges['geometry'].apply(lambda x: len(x.coords))

# エッジの距離とポイント数から曲線率?を求める。
gdf_edges['curvature'] = gdf_edges['point_cnt'] / gdf_edges['length']

# 3座標間の角度を求める
def calculate_angle_between_vectors(A, B, C):
    vector_AB = np.array(B) - np.array(A)
    vector_BC = np.array(C) - np.array(B)
    
    dot_product = np.dot(vector_AB, vector_BC)
    norm_AB = np.linalg.norm(vector_AB)
    norm_BC = np.linalg.norm(vector_BC)
    
    cosine_theta = dot_product / (norm_AB * norm_BC)
    angle_rad = np.arccos(cosine_theta)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

# 座標間の角度の変化の合計値を求める
gdf_edges['geometory_angle_total'] = gdf_edges['geometry'].apply(
    lambda x: sum([calculate_angle_between_vectors(x.coords[i-1], x.coords[i], x.coords[i+1]) for i in range(1, len(x.coords)-1)])
)

# 抽出対象のエッジにフラグを立てる.条件はcurvatureが0.025より上 かつ length_evaluationが0.5以上
gdf_edges['is_target'] = np.where((gdf_edges['curvature'] > 0.025) & (gdf_edges['length_evaluation'] > 0.5) & (gdf_edges['geometory_angle_total'] > 120), 1, 0)

# 距離と角度から評価値を計算する
gdf_edges['evaluate'] = gdf_edges['length_evaluation'] * gdf_edges['geometory_angle_total']

# 開始位置列を追加する
gdf_edges['start_point'] = gdf_edges['geometry'].apply(lambda x: x.coords[0])
# 終了位置列を追加する
gdf_edges['end_point'] = gdf_edges['geometry'].apply(lambda x: x.coords[-1])


# 逆方向のベクトルを持つエッジを削除する
gdf_edges = utils.drop_duplicate_edge(gdf_edges)

target_edges = gdf_edges[gdf_edges['is_target'] == 1]

# targe_edgesを並び替える
target_edges = target_edges.sort_values(['evaluate'], ascending=[False])

# 地図を表示する
map_osm = ox.plot_graph_folium(graph, edge_width=2)


folium_utils.add_marker(map_osm, latitude_start, longitude_start, "my-house", "red")
folium_utils.add_marker(map_osm, latitude_end, longitude_end, "rz-factory", "green")

# # is_targetが正のエッジにplotを立てて緯度と経度を表示する
# for index, row in gdf_edges[gdf_edges['is_target'] == 1].iterrows():
#     folium_utils.add_marker(map_osm,
#                             row['geometry'].coords[0][1],
#                             row['geometry'].coords[0][0],
#                             f"{row['geometry'].coords[0][1]}, {row['geometry'].coords[0][0]}, {row['geometory_angle_total']}, {row['length']}",
#                             "red")

# 候補を全て表示
map_osm.add_child(
    folium.features.GeoJson(
        gdf_edges[gdf_edges['is_target'] == 1].to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#0000ff',
            'fill_opacity': 0,
            'weight': 2,
        }
    )
)

# 候補の上位10件を表示
map_osm.add_child(
    folium.features.GeoJson(
        target_edges.head(15).to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#ff0000',
            'fill_opacity': 0,
            'weight': 5,
        }
    )
)

